In [13]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np

from utils.timescale_connector import TimescaleConnector
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import lightning as L
import mlflow 
from mlflow.models import infer_signature

In [3]:
# Data
df = TimescaleConnector.query_ohlcv_daily("ACB")

In [4]:
df.head()

,date,open,high,low,close,volume,symbol
0,2017-01-03,4.43,4.75,4.43,4.75,1609757,ACB
1,2017-01-04,4.75,4.88,4.75,4.78,739000,ACB
2,2017-01-05,4.80,4.83,4.73,4.78,471881,ACB
3,2017-01-06,4.78,5.23,4.73,5.03,1409824,ACB
4,2017-01-09,5.03,5.23,5.03,5.20,970791,ACB


In [10]:
df

,date,open,high,low,close,volume,symbol
0,2017-01-03,4.43,4.75,4.43,4.75,1609757,ACB
1,2017-01-04,4.75,4.88,4.75,4.78,739000,ACB
2,2017-01-05,4.80,4.83,4.73,4.78,471881,ACB
3,2017-01-06,4.78,5.23,4.73,5.03,1409824,ACB
4,2017-01-09,5.03,5.23,5.03,5.20,970791,ACB
...,...,...,...,...,...,...,...
1675,2023-09-25,22.30,22.35,22.00,22.00,7623000,ACB
1676,2023-09-26,22.05,22.25,21.85,21.90,9404200,ACB
1677,2023-09-27,21.90,22.00,21.75,22.00,5685000,ACB
1678,2023-09-28,21.85,22.00,21.70,21.75,4991800,ACB


In [12]:
X = df.drop(columns=["close", "date", "symbol"])
X

,open,high,low,volume
0,4.43,4.75,4.43,1609757
1,4.75,4.88,4.75,739000
2,4.80,4.83,4.73,471881
3,4.78,5.23,4.73,1409824
4,5.03,5.23,5.03,970791
...,...,...,...,...
1675,22.30,22.35,22.00,7623000
1676,22.05,22.25,21.85,9404200
1677,21.90,22.00,21.75,5685000
1678,21.85,22.00,21.70,4991800


In [ ]:
# Creat a data module
class OhlcvDataModule(L.LightningDataModule):
    def __init__(self, seq_len=1, batch_size=128, num_workers=16) -> None:
        super().__init__()
        self.seq_len = seq_len
        self.batch_size = batch_size
        self.num_workers = num_workers
        
    def prepare_data(self):
        # Get data
        df = TimescaleConnector.query_ohlcv_daily("ACB")
        
        # Drop date and symbol columns
        df = df.drop(columns=["date", "symbol"])
        
        # Change data types to float
        df = df.astype(float)
        
        # Define features and target, X and y
        X = df.drop(columns=["close"]).values
        y = df["close"].values
        
        # Split data into train, validation, and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, shuffle=False)
        
        # Scale data
        self.scaler = StandardScaler()
        X_train = self.scaler.fit_transform(X_train)
        X_val = self.scaler.transform(X_val)
        X_test = self.scaler.transform(X_test)
        

    

**Define the model with Lightning**